![alt text](https://drive.google.com/uc?export=view&id=1UXScsVx_Wni_JuDdB8LeTnM6jsPfIwkW)

Proprietary content. © Great Learning. All Rights Reserved. Unauthorized use or distribution prohibited.

# Sentiment Classification

### Dataset
- Dataset of 50,000 movie reviews from IMDB, labeled by sentiment positive (1) or negative (0)
- Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers).
- For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".
- As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.

Command to import data
- `from tensorflow.keras.datasets import imdb`

### Import the data (2 Marks)
- Use `imdb.load_data()` method
- Get train and test set
- Take 10000 most frequent words

In [100]:
#### Add your code here ####
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding,TimeDistributed,Flatten
from keras.layers import LSTM
from keras.datasets import imdb
import tensorflow

In [101]:
max_features = 10000
maxlen = 300  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


### Pad each sentence to be of same length (2 Marks)
- Take maximum sequence length as 300

In [102]:
#### Add your code here ####
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

Pad sequences (samples x time)


### Print shape of features & labels (2 Marks)

Number of review, number of words in each review

In [103]:
#### Add your code here ####
x_train_shape=x_train.shape
x_test_shape=x_test.shape
print('x_train shape:', x_train_shape)
print('x_test shape:', x_test_shape)

x_train shape: (25000, 300)
x_test shape: (25000, 300)


In [104]:
#### Add your code here ####

print('Number of train review:', x_train_shape[0])
print('Number of test review:', x_test_shape[0])

Number of train review: 25000
Number of test review: 25000


In [105]:

print('Number of train words in each review:', x_train_shape[1])
print('Number of test words in each review:', x_test_shape[1])

Number of train words in each review: 300
Number of test words in each review: 300


Number of labels

In [106]:
#### Add your code here ####

print('y_train Number of train labels:', y_train.shape)

y_train Number of train labels: (25000,)


In [107]:
#### Add your code here ####

print('y_test Number of train labels:', y_test.shape)

y_test Number of train labels: (25000,)


### Print value of any one feature and it's label (2 Marks)

Feature value

In [108]:
#### Add your code here ####
print('Movie Review', x_train[0])

Movie Review [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    1   14
   22   16   43  530  973 1622 1385   65  458 4468   66 3941    4  173
   36  256    5   25  100   43  838  112   50  670    2    9   35  480
  284    5  150    4  172  112  167    2  336  385   39    4  172 4536
 1111   17  546   38   13  447    4  192   50   16    6  147 2025   19
   14   22    4 1920 4613  469    4   22   71   87   12   16   43  530
   38   76   15   13 1247    4   22   17  515   17   12   16  626   18
    2    5   62  386   12    8  316    8  106    5    4 2223 5244   16
  480   66 3785   33    4  130   12   16   38  619    5   25  12

Label value

In [109]:
#### Add your code here ####
print('Sentiment', y_train[0])

Sentiment 1


### Decode the feature value to get original sentence (2 Marks)

First, retrieve a dictionary that contains mapping of words to their index in the IMDB dataset

In [110]:
#### Add your code here ####
index=tensorflow.keras.datasets.imdb.get_word_index()

Now use the dictionary to get the original words from the encodings, for a particular sentence

In [111]:
#### Add your code here ####
reverse_index = dict([(value, key) for (key, value) in index.items()]) 
decoded = " ".join( [reverse_index.get(i -3, "#") for i in x_train[0]] )
print('Decoded Movie Review')
print(decoded)

Decoded Movie Review
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert # is an amazing actor and now the same being director # father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for # and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also # to the two little boy's that played the # of norman and paul they were just brilliant children are often left out of the # list i

Get the sentiment for the above sentence
- positive (1)
- negative (0)

In [112]:
#### Add your code here ####
label_test=y_train[0]
print('Sentiment:')
if label_test==1:
  print('positive')
else:
  print('negative')


Sentiment:
positive


### Define model (10 Marks)
- Define a Sequential Model
- Add Embedding layer
  - Embedding layer turns positive integers into dense vectors of fixed size
  - `tensorflow.keras` embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unique integer number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn LabelEncoder.
  - Size of the vocabulary will be 10000
  - Give dimension of the dense embedding as 100
  - Length of input sequences should be 300
- Add LSTM layer
  - Pass value in `return_sequences` as True
- Add a `TimeDistributed` layer with 100 Dense neurons
- Add Flatten layer
- Add Dense layer

In [113]:
#### Add your code here ####
print('Build model...')
model = Sequential()
model.add(Embedding(max_features,100, input_length=maxlen))
model.add(Dense(100, activation='relu'))
model.add(LSTM(108, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(TimeDistributed(Dense(100)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

Build model...


### Compile the model (2 Marks)
- Use Optimizer as Adam
- Use Binary Crossentropy as loss
- Use Accuracy as metrics

In [114]:
#### Add your code here ####
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### Print model summary (2 Marks)

In [115]:
#### Add your code here ####
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 300, 100)          1000000   
_________________________________________________________________
dense_10 (Dense)             (None, 300, 100)          10100     
_________________________________________________________________
lstm_5 (LSTM)                (None, 300, 108)          90288     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 300, 100)          10900     
_________________________________________________________________
flatten_2 (Flatten)          (None, 30000)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 30001     
Total params: 1,141,289
Trainable params: 1,141,289
Non-trainable params: 0
____________________________________________

### Fit the model (2 Marks)

In [116]:
#### Add your code here ####
print('Train...')
model.fit(x_train, y_train,
          batch_size=256,
          epochs=10,
          validation_data=(x_test, y_test))


Train...
Epoch 1/10
98/98 [==============================] - 166s 2s/step - loss: 0.4448 - accuracy: 0.7663 - val_loss: 0.3049 - val_accuracy: 0.8672
Epoch 2/10
98/98 [==============================] - 166s 2s/step - loss: 0.2181 - accuracy: 0.9148 - val_loss: 0.2909 - val_accuracy: 0.8788
Epoch 3/10
98/98 [==============================] - 165s 2s/step - loss: 0.1630 - accuracy: 0.9378 - val_loss: 0.3145 - val_accuracy: 0.8695
Epoch 4/10
98/98 [==============================] - 165s 2s/step - loss: 0.1278 - accuracy: 0.9518 - val_loss: 0.4132 - val_accuracy: 0.8596
Epoch 5/10
98/98 [==============================] - 164s 2s/step - loss: 0.1006 - accuracy: 0.9624 - val_loss: 0.4634 - val_accuracy: 0.8551
Epoch 6/10
98/98 [==============================] - 164s 2s/step - loss: 0.0817 - accuracy: 0.9697 - val_loss: 0.5968 - val_accuracy: 0.8471
Epoch 7/10
98/98 [==============================] - 164s 2s/step - loss: 0.0659 - accuracy: 0.9757 - val_loss: 0.6445 - val_accuracy: 0.8486
Epoc

### Evaluate model (2 Marks)

In [117]:
#### Add your code here ####
score, acc = model.evaluate(x_test, y_test,
                            batch_size=256)
print('Test score:', score)
print('Test accuracy:', acc)

98/98 [==============================] - 9s 93ms/step - loss: 1.0843 - accuracy: 0.8403
Test score: 1.0842998027801514
Test accuracy: 0.8403199911117554


### Predict on one sample (2 Marks)

In [127]:
#### Add your code here ####
n=1
review=x_test[n:n+1]

In [128]:
sentiment=model.predict(review)[0]
x_test

array([[   0,    0,    0, ...,   14,    6,  717],
       [   0,    0,    0, ...,  125,    4, 3077],
       [1239, 5189,  137, ...,    9,   57,  975],
       ...,
       [   0,    0,    0, ...,   21,  846, 5518],
       [   0,    0,    0, ..., 2302,    7,  470],
       [   0,    0,    0, ...,   34, 2005, 2643]], dtype=int32)

In [129]:
print('Pridictive Value:')
if(float(sentiment[0])<=0.5):
  print("negative(0)")
else:
  print("positive(1)")

Pridictive Value:
positive(1)


In [130]:
print("Real/Given Value:")
print(y_test[n])

Real/Given Value:
1
